In [36]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input, add, dot, Flatten, Embedding,Dropout, concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers

In [37]:
def split_dict_to_multiple(input_dict, max_limit):
    """Splits dict into multiple dicts with given maximum size.
        Returns a list of dictionaries."""
    chunks = []
    curr_dict = {"user" : [], "item" : [], "date" : [], "rating" : [] }
    for i in range(0,int(len(input_dict["item"]))):
        curr_dict["date"].append(ratings_dict["date"][i])
        curr_dict["item"].append(ratings_dict["item"][i])
        curr_dict["user"].append(ratings_dict["user"][i])
        curr_dict["rating"].append(ratings_dict["rating"][i])

        if i != 0 and i % int(len(input_dict["item"])/max_limit) == 0:
            chunks.append(curr_dict)
            curr_dict = {"user" : [], "item" : [], "date" : [], "rating" : [] }

        
    return chunks

In [38]:
from db_handler import DBHandler

db = DBHandler()
db.setup_test_table()
user_info = {}
ratings_dict = {"user" : [], "item" : [], "date" : [], "rating" : [] }
for x in db.read_data("comp3208-train-big.csv"):
    listParts = x.strip().split(',')
    if listParts[2] == "rating":
        continue
    ratings_dict["item"].append(int(listParts[1]))
    ratings_dict["user"].append(int(listParts[0]))
    ratings_dict["date"].append(float(listParts[3]))
    ratings_dict["rating"].append(float(listParts[2]))
#     if listParts[0] in user_info:
#         user_info[listParts[0]][listParts[1]] = float(listParts[2])
#     else:
#         user_info[listParts[0]] = {listParts[1]: float(listParts[2])}

rating_chunks = split_dict_to_multiple(ratings_dict,15)
df = pd.DataFrame(ratings_dict)
df

INFO 2021-03-02 13:38:54,042 logging started
INFO 2021-03-02 13:38:54,051 Setting up test table
INFO 2021-03-02 13:38:55,102 Reading data from database: comp3208-train-big.csv


,user,item,date,rating
0,1,12793,1.252961e+09,4.0
1,1,31216,1.252964e+09,4.0
2,1,38744,1.252963e+09,3.5
3,1,113,1.252964e+09,1.5
4,1,33000,1.252965e+09,3.0
...,...,...,...,...
26365767,283229,27303,1.017721e+09,4.0
26365768,283229,6853,1.017722e+09,5.0
26365769,283229,19353,1.017720e+09,4.0
26365770,283229,37694,1.017721e+09,4.0


In [39]:
db = DBHandler()
test_ratings_dict = {"user" : [], "item" : [], "date" : []}
for x in db.read_data("comp3208-test-big.csv"):
    listParts = x.strip().split(',')
    if listParts[2] == "rating":
        continue
    test_ratings_dict["item"].append(int(listParts[1]))
    test_ratings_dict["user"].append(int(listParts[0]))
    test_ratings_dict["date"].append(float(listParts[2]))
    
test_df = pd.DataFrame(test_ratings_dict)
test_df

INFO 2021-03-02 13:39:59,021 logging started
INFO 2021-03-02 13:39:59,248 Reading data from database: comp3208-test-big.csv


,user,item,date
0,1,18953,1.252961e+09
1,1,47984,1.252965e+09
2,1,42356,1.252963e+09
3,1,14254,1.252963e+09
4,1,27049,1.252964e+09
...,...,...,...
1387667,283226,32255,8.628093e+08
1387668,283226,499,8.628105e+08
1387669,283228,27278,9.747583e+08
1387670,283228,19203,9.747582e+08


In [40]:
item_reference = {}
counter = 0
for line in df.itertuples():
    if line[2] not in item_reference:
        item_reference[line[2]] = counter
        counter += 1
        
print("finished item")

person_reference = {}
counter = 0
for line in df.itertuples():
    if line[1] not in person_reference:
        person_reference[line[1]] = counter
        counter += 1
        
date_reference = {}
counter = 0
for line in df.itertuples():
    if line[3] not in date_reference:
        date_reference[line[3]] = counter
        counter += 1

finished item


In [41]:
print(len(item_reference), len(person_reference),len(date_reference))

53347 282950 24876869


In [42]:
# Normalising dates 
ratings = df.copy()
test_ratings = test_df.copy()
cols_to_norm = ['date']
ratings[cols_to_norm] = ratings[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
test_ratings[cols_to_norm] = test_ratings[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
ratings,test_ratings

(            user   item      date  rating
 0              1  12793  0.619155     4.0
 1              1  31216  0.619159     4.0
 2              1  38744  0.619157     3.5
 3              1    113  0.619159     1.5
 4              1  33000  0.619160     3.0
 ...          ...    ...       ...     ...
 26365767  283229  27303  0.304787     4.0
 26365768  283229   6853  0.304787     5.0
 26365769  283229  19353  0.304785     4.0
 26365770  283229  37694  0.304786     4.0
 26365771  283229  13849  0.304787     3.0
 
 [26365772 rows x 4 columns],
            user   item      date
 0             1  18953  0.601461
 1             1  47984  0.601467
 2             1  42356  0.601463
 3             1  14254  0.601463
 4             1  27049  0.601465
 ...         ...    ...       ...
 1387667  283226  32255  0.055849
 1387668  283226    499  0.055851
 1387669  283228  27278  0.212406
 1387670  283228  19203  0.212406
 1387671  283229  29096  0.272488
 
 [1387672 rows x 3 columns])

In [43]:
def index_user_and_items(item_col, user_col):
    """
    Indexes the user and items 
    """
    
    unique_users = ratings[user_col].append(test_ratings[user_col]).unique()
    user_index = {user:i for i,user in enumerate(unique_users)}
    ratings[user_col] = ratings[user_col].apply(lambda user: user_index[user])
    test_ratings[user_col] = test_ratings[user_col].apply(lambda user: user_index[user])
    n_users = len(unique_users)
        
    unique_item = ratings[item_col].append(test_ratings[item_col]).unique()
    item_index = {item:i for i,item in enumerate(unique_item)}
    ratings[item_col] = ratings[item_col].apply(lambda item: item_index[item])
    test_ratings[item_col] = test_ratings[item_col].apply(lambda item: item_index[item])

    n_items=len(unique_item)
    
    return n_users, n_items

n_users, n_items = index_user_and_items("item",'user')
max_timestamp = df["date"].max()
min_timestamp = df["date"].min()

max_timestamp,min_timestamp,n_users, n_items

(1537945770.0, 789651240.0, 283228, 53889)

In [44]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings,test_size=0.25)

In [45]:
from sklearn.model_selection import train_test_split

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
train, test = train_test_split(ratings, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
val, test = train_test_split(test, test_size=test_ratio/(test_ratio + validation_ratio)) 

print(train, val, test)

            user  item      date  rating
25673858  275568  2750  0.247158     3.0
12874415  138100     8  0.243353     4.0
23254882  249554   100  0.846943     4.5
1820075    20153  2731  0.200342     3.0
21323458  228721  1446  0.128282     3.0
...          ...   ...       ...     ...
9410061   101037   951  0.303091     3.0
25830687  277145  1080  0.419352     4.0
208217      2341  1741  0.941246     3.0
25076317  269043   444  0.254065     3.0
16231563  173723  7669  0.455013     2.5

[19774329 rows x 4 columns]             user  item      date  rating
12374044  133198  2215  0.678046     4.0
976195     10762   924  0.406233     1.0
12770325  137134   637  0.056672     4.0
11524174  123764  3496  0.940680     4.0
17721927  189988  1073  0.439714     2.5
...          ...   ...       ...     ...
22083639  236598   736  0.079214     1.0
11741316  126359   830  0.532288     2.0
17398889  186487   358  0.384076     3.0
10071670  108065  1348  0.770076     1.5
4137789    45250   188  0.19

In [46]:
# https://medium.com/@jdwittenauer/deep-learning-with-keras-recommender-systems-e7b99cb29929
# https://www.tensorflow.org/recommenders/examples/featurization
from keras.layers import Concatenate, Dense, Dropout
from keras.layers import Add, Activation, Lambda
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam,Nadam,RMSprop
from keras.regularizers import l2

# Layer to abstract away the keras embedding
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        # Creates the emedding layer and reshapes it
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x
    
    
def CreatePredictionModel(x_train, y_train, x_val, y_val, params,min_rating=0.5, max_rating=5):
    # User embedding
    user = Input(shape=(1,))
    u = EmbeddingLayer(params["n_users"], params["user_factors"])(user)
    
    # Item embedding
    item = Input(shape=(1,))
    i = EmbeddingLayer(params["n_items"], params["item_factors"])(item)
    
    
    # Date embedding
    date = Input(shape=(1,))
    timestamp_buckets = np.linspace(min_timestamp, max_timestamp, num=1000)
    d = EmbeddingLayer(len(timestamp_buckets), params["time_stamp_factors"])(date)
    
    
    # Concat them together and put a fully connected layer on top
    x = Concatenate()([u, i,d])
    x = Dropout(params["f-dropout"])(x)
    
    x = Dense(params['first_neuron'], kernel_initializer='he_normal')(x)
    x = Activation(params['activation'])(x)
    # Added dropout layer to avoid overfitting
    x = Dropout(params['s-dropout'])(x)
    
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    # Function to map to result
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, item, date], outputs=x)
    
    # Using Adam optimiser
    opt = Adam(lr=params['lr'])
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    # 0 is user, 1 is item, 2 is date
    es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    history = model.fit(x=[x_train[:,0], x_train[:,1],x_train[:,2]],
                        y = y_train,
                        batch_size=params["batch_size"], 
                        epochs=params["epochs"],
                        validation_data=([x_val[:,0], x_val[:,1],x_val[:,2]], y_val ))
    
    
    return history, model

In [214]:
p1 = {
    'user_factors' : [10,30,50,100],
    "n_users" : [n_users],
    "n_items" : [n_items],
    "item_factors": [10,30,50,100],
    "time_stamp_factors" : [16,32,64],
    'first_neuron':[4, 8, 16, 32, 64],
    "batch_size": (16,32,64),
    "epochs" : [5,10,15,25],
    'f-dropout': (0,0.05, 0.5),
    's-dropout': (0,0.1,0.5, 5),
    'activation':['relu', 'elu'],
    'lr': (0.001, 0.01, 1)
}

In [215]:
# import talos as ta
# t = ta.Scan(train[['user','item','date']].to_numpy(),
#             train['rating'].to_numpy(),
#             model=RecommenderNet,
#             fraction_limit=0.1, 
#             params=p,
#             experiment_name='1')

In [158]:
# # Analyse result
# r = ta.Reporting(t)
# # Show the models
# display(r.data.sort_values(by=['val_loss']))

,start,end,duration,round_epochs,loss,val_loss,activation,batch_size,epochs,f-dropout,first_neuron,item_factors,lr,n_items,n_users,s-dropout,time_stamp_factors,user_factors
605,02/25/21-194930,02/25/21-194954,24.238501,10,0.698704,0.794792,elu,20,15,0.0,8,50,0.001,9593,18729,0.0,16,30
244,02/25/21-180307,02/25/21-180327,20.025503,10,0.683020,0.795099,elu,31,10,0.0,4,50,0.001,9593,18729,0.0,32,30
442,02/25/21-185649,02/25/21-185717,27.417002,10,0.687680,0.795102,elu,31,10,0.0,16,50,0.001,9593,18729,0.0,16,50
227,02/25/21-175840,02/25/21-175905,24.150000,10,0.701118,0.795213,elu,21,10,0.0,32,30,0.001,9593,18729,0.0,64,50
594,02/25/21-194520,02/25/21-194545,25.807999,10,0.691700,0.795442,elu,30,10,0.0,16,10,0.001,9593,18729,0.0,32,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,02/25/21-181704,02/25/21-181725,20.611079,10,0.804617,0.835850,relu,17,10,0.0,4,10,0.001,9593,18729,0.3,32,50
85,02/25/21-171429,02/25/21-171439,10.036503,10,0.810203,0.838397,relu,18,15,0.0,4,30,0.001,9593,18729,0.2,64,10
71,02/25/21-170936,02/25/21-170942,5.640499,10,0.820545,0.842583,relu,17,10,0.0,4,10,0.001,9593,18729,0.3,64,10
174,02/25/21-174332,02/25/21-174338,5.537501,10,0.818880,0.843087,relu,26,10,0.0,4,10,0.001,9593,18729,0.3,32,10


In [ ]:
# # Second run
# t = ta.Scan(train[['user','item','date']].to_numpy(),
#             train['rating'].to_numpy(),
#             model=CreatePredictionModel,
#             fraction_limit=0.01, 
#             params=p,
#             experiment_name='1')

In [ ]:
final_param = {
    'user_factors' : 50,
    "n_users" : n_users,
    "n_items" : n_items,
    "item_factors": 50,
    "time_stamp_factors" : 50,
    'first_neuron':32,
    "batch_size": 32000,
    "epochs" : 20,
    'f-dropout': 0.01,
    's-dropout': 0.4,
    'activation':'relu',
    'lr': 0.001
}

h, model = CreatePredictionModel(ratings[['user','item','date']].to_numpy(),
               ratings['rating'].to_numpy(), 
               val[['user','item','date']].to_numpy(),
               val['rating'].to_numpy(),
              final_param)

Epoch 1/20
824/824 [==============================] - 135s 161ms/step - loss: 0.9249 - val_loss: 0.7235
Epoch 2/20
824/824 [==============================] - 130s 158ms/step - loss: 0.7550 - val_loss: 0.7040
Epoch 3/20
824/824 [==============================] - 133s 162ms/step - loss: 0.7340 - val_loss: 0.6889
Epoch 4/20
169/824 [=====>........................] - ETA: 1:41 - loss: 0.7147

In [ ]:
# Final result
predicted = model.predict([test.user,test.item,test.date])
predicted
predicted_round = [round(x[0] * 2) / 2 for x in predicted]
predicted_round
from sklearn.metrics import mean_squared_error
print("MSE for test set: ", mean_squared_error(list(test.rating), predicted))

In [ ]:
predicted = model.predict([test_ratings.user,test_ratings.item,test_ratings.date])

In [ ]:
final_dict = {"user" : [], "item" : [], "date" : [], "rating" : [] }
for i,line in enumerate(test_df.itertuples()):
    final_dict["item"].append(line[2])
    final_dict["user"].append(line[1])
    final_dict["date"].append(line[3])
    final_dict["rating"].append(predicted[i][0])

    
final_df = pd.DataFrame(final_dict)
final_df